In [32]:
import pandas as pd ; import networkx as nx ; import ast ; import matplotlib.pyplot as plt ; import numpy as np ;
import nltk ; from wordcloud import WordCloud, ImageColorGenerator ; from PIL import Image ; import urllib.request

In [56]:
df_characters = pd.read_csv("RaM_characters_wRelationsAndAttributes.csv")
df_characters = df_characters.drop('Unnamed: 0', axis=1)
df_episodes = pd.read_csv("RaM_episodes.csv")

In [57]:
df_characters

,Name,Hyperlink,Relations,Species,Origin,Gender,Status
0,Abandoned Jerrys,https://rickandmorty.fandom.com/wiki/Abandoned...,['Jerry Smith'],unknown,unknown,Genderless,unknown
1,Abrodolph Lincoler,https://rickandmorty.fandom.com/wiki/Abrodolph...,"['Rick Sanchez', 'Brad', 'Morty Smith', 'Nancy']",unknown,unknown,Genderless,unknown
2,Adam,https://rickandmorty.fandom.com/wiki/Adam,[],Humanoid,Narnia Dimension,Male,Dead
3,Adjudicator Rick,https://rickandmorty.fandom.com/wiki/Adjudicat...,['Evil Morty'],Human,unknown,Male,Dead
4,Afro Rick,https://rickandmorty.fandom.com/wiki/Afro_Rick,"['Rick Sanchez', 'Morty Smith', 'Council of Ri...",unknown,unknown,Genderless,unknown
...,...,...,...,...,...,...,...
534,Zarbadar's Mytholog,https://rickandmorty.fandom.com/wiki/Zarbadar%...,"['Gar Gloonch', 'Zarbadar Gloonch']",Mythological Creature,Nuptia 4,Female,unknown
535,Zeep Xanflorp,https://rickandmorty.fandom.com/wiki/Zeep_Xanf...,"['Rick Sanchez', 'Chris (The Ricks Must Be Cra...",Humanoid,Rick's Battery Microverse,Male,Alive
536,Zerillians,https://rickandmorty.fandom.com/wiki/Zerillians,['Jerry Smith'],unknown,unknown,Genderless,unknown
537,Zeta Alpha Rick,https://rickandmorty.fandom.com/wiki/Zeta_Alph...,"['Council of Ricks', 'Rick Sanchez', 'Rick Pri...",Human,unknown,Male,Dead


In [58]:
df_episodes

,Title,Hyperlink,Season_nb,Episode_nb
0,Pilot,https://rickandmorty.fandom.com/wiki/Pilot,1,1
1,Lawnmower Dog,https://rickandmorty.fandom.com/wiki/Lawnmower...,1,2
2,Anatomy Park (Episode),https://rickandmorty.fandom.com/wiki/Anatomy_P...,1,3
3,M. Night Shaym-Aliens!,https://rickandmorty.fandom.com/wiki/M._Night_...,1,4
4,Meeseeks and Destroy,https://rickandmorty.fandom.com/wiki/Meeseeks_...,1,5
5,Rick Potion 9,https://rickandmorty.fandom.com/wiki/Rick_Poti...,1,6
6,Raising Gazorpazorp,https://rickandmorty.fandom.com/wiki/Raising_G...,1,7
7,Rixty Minutes,https://rickandmorty.fandom.com/wiki/Rixty_Min...,1,8
8,Something Ricked This Way Comes,https://rickandmorty.fandom.com/wiki/Something...,1,9
9,Close Rick-counters of the Rick Kind,https://rickandmorty.fandom.com/wiki/Close_Ric...,1,10


In [83]:
df_transcripts = pd.read_csv("https://raw.githubusercontent.com/ThomaSpyrou/SocialGraphsAss1/main/Final%20Project%20-%20Rick%20and%20Morty/transcripts/RickAndMortyScripts.csv")
df_transcripts = df_transcripts.drop('index', axis = 1)
df_transcripts

,season no.,episode no.,episode name,name,line
0,1,1,Pilot,Rick,Morty! You gotta come on. Jus'... you gotta co...
1,1,1,Pilot,Morty,"What, Rick? What’s going on?"
2,1,1,Pilot,Rick,"I got a surprise for you, Morty."
3,1,1,Pilot,Morty,It's the middle of the night. What are you tal...
4,1,1,Pilot,Rick,"Come on, I got a surprise for you. Come on, h..."
...,...,...,...,...,...
1900,3,7,Tales From the Citadel,Morty,That was amazing!
1901,3,7,Tales From the Citadel,Rick,Got some of that mermaid puss!
1902,3,7,Tales From the Citadel,Morty,I'm really hoping it wasn't a one-off thing an...
1903,3,7,Tales From the Citadel,Rick,"Pssh! Not at all, Morty. That place will never..."


In [62]:
df_charLines = df_transcripts.groupby(['name'], as_index = False).agg({'line': ','.join})
df_charLines

,name,line
0,Agency Director,"What is it?,Where is it?,We have 34 armed guar..."
1,Alan,I wish he had the ability to check is attitude...
2,Alien Doctor,"Relax Mr. Smith, you're in an alien hospital. ..."
3,All Mortys,"Oh man... I don't feel good.,Wait, who said th..."
4,All Ricks,"Oh God, oh no, what did you guys just do?,Alri..."
5,All Summers,"Wait, what am I saying?,What's happening Gramp..."
6,Announcer,The glarp zone is for flarping and unglarping ...
7,Beth,"Morty, are you getting sick? I told you not t..."
8,Birdperson,"Morty.,You appear to be dying. I will make eff..."
9,Brad,"What are you doing?,Wait, wait, Were you about..."


In [63]:
characters_dialogue = df_charLines.set_index('name').T.to_dict('list') 
characters_dialogue

{'Agency Director': ['What is it?,Where is it?,We have 34 armed guards, and we can\'t kill a pickle?,Put him on, and locate the phone he\'s using.,Your mere presence in this building violates international law.,And how is that?,Shut your mouth and do your jobs, you fucking children!,Some of my men are calling you "Solen\'ya",Ah.,He\'s just a pickle!,He\'s not the only one.,Then Katarina is a prisoner. Perhaps I could arrange her escape, as well. She lives, Jaguar.,Do it. Okay, you win, Pickle Man. I\'m unsealing the building.,Pickle Man, there\'s $100 million worth of bonds in a safe on level two. I\'ll give you the combination.,Shut up and call me a helicopter, you prick! Do we have a deal?!,Jaguar\'s daughter is dead.,Jaguar was an animal. You\'re an intelligent pickle. We can do business.,Is the helicopter here?,Tell them we were robbed.,Farewell, Solen\'ya.'],
 'Alan': ['I wish he had the ability to check is attitude.,All aboard!,I could\'ve just used a ghost train.,Man, fuck you.,

### LabMT method

In [64]:
url_LabMT = "https://raw.githubusercontent.com/ThomaSpyrou/SocialGraphsAss1/main/Final%20Project%20-%20Rick%20and%20Morty/dataset_LabMT"

LabMT = urllib.request.urlopen(url_LabMT).read().decode('utf-8')
# remove the useless start of the text
LabMT = LabMT[172:]
# process text file for dictionary conversion
LabMT = LabMT.split('\n')
LabMT = [aList.split('\t') for aList in LabMT]
# create LabMT dictionary
LabMT_dict = {}
for aList in LabMT:
    if len(aList) == 8:
        aWord = aList[0]
        # we only take "happiness_average" in account here
        aHappinessAvg = float(aList[2])
        LabMT_dict[aWord]=aHappinessAvg

In [65]:
from nltk.tokenize import RegexpTokenizer

def get_list_of_tokens(aSentence):
    """ Transform a sentence (string) into a list of tokens """
    # Set to lower case
    aSentence = aSentence.lower()
    # Tokenize
    tokens = RegexpTokenizer(r'\w+').tokenize(aSentence)
    # Lemmatize words
    tokens = [ nltk.WordNetLemmatizer().lemmatize(w) for w in tokens ]
    return tokens

def get_happiness_average(aWord):
    """ Get the happiness average of a word from LabMT dictionary """
    if aWord in list(LabMT_dict.keys()) :
        return LabMT_dict[aWord]
    return None

def calculate_LabMT_sentiment(aListOfTokens):
    """ Calculate the sentiment score of a list of tokens """
    sentiment_score = 0
    if aListOfTokens != []:
        for aToken in aListOfTokens:
            anHapinessAvg = get_happiness_average(aToken)
            if anHapinessAvg != None:
                sentiment_score += anHapinessAvg
        return sentiment_score/len(aListOfTokens)
    return sentiment_score

In [66]:
# calculate the LabMT sentiment score for each character

sentiment_LabMT_score_list = []

def calculate_LabMT_sentiment_of_a_node(aRow):
    if aRow.Name in list(characters_dialogue.keys()):
        sentiment_LabMT_score_list.append(np.mean([calculate_LabMT_sentiment(get_list_of_tokens(aSentence)) for aSentence in characters_dialogue[aRow.Name]]))
    else:
        sentiment_LabMT_score_list.append(None)

_ = df_characters.apply(calculate_LabMT_sentiment_of_a_node, axis=1)

In [67]:
# store in dataframe of characters as a new column
df_characters['LabMT_score']=sentiment_LabMT_score_list
df_LabMT = df_characters[df_characters.LabMT_score.notnull()]

In [68]:
df_characters

,Name,Hyperlink,Relations,Species,Origin,Gender,Status,LabMT_score
0,Abandoned Jerrys,https://rickandmorty.fandom.com/wiki/Abandoned...,['Jerry Smith'],unknown,unknown,Genderless,unknown,NaN
1,Abrodolph Lincoler,https://rickandmorty.fandom.com/wiki/Abrodolph...,"['Rick Sanchez', 'Brad', 'Morty Smith', 'Nancy']",unknown,unknown,Genderless,unknown,NaN
2,Adam,https://rickandmorty.fandom.com/wiki/Adam,[],Humanoid,Narnia Dimension,Male,Dead,NaN
3,Adjudicator Rick,https://rickandmorty.fandom.com/wiki/Adjudicat...,['Evil Morty'],Human,unknown,Male,Dead,NaN
4,Afro Rick,https://rickandmorty.fandom.com/wiki/Afro_Rick,"['Rick Sanchez', 'Morty Smith', 'Council of Ri...",unknown,unknown,Genderless,unknown,NaN
...,...,...,...,...,...,...,...,...
534,Zarbadar's Mytholog,https://rickandmorty.fandom.com/wiki/Zarbadar%...,"['Gar Gloonch', 'Zarbadar Gloonch']",Mythological Creature,Nuptia 4,Female,unknown,NaN
535,Zeep Xanflorp,https://rickandmorty.fandom.com/wiki/Zeep_Xanf...,"['Rick Sanchez', 'Chris (The Ricks Must Be Cra...",Humanoid,Rick's Battery Microverse,Male,Alive,NaN
536,Zerillians,https://rickandmorty.fandom.com/wiki/Zerillians,['Jerry Smith'],unknown,unknown,Genderless,unknown,NaN
537,Zeta Alpha Rick,https://rickandmorty.fandom.com/wiki/Zeta_Alph...,"['Council of Ricks', 'Rick Sanchez', 'Rick Pri...",Human,unknown,Male,Dead,NaN


In [69]:
df_LabMT

,Name,Hyperlink,Relations,Species,Origin,Gender,Status,LabMT_score
64,Brad,https://rickandmorty.fandom.com/wiki/Brad,"['Jessica', 'Morty Smith', 'Summer Smith', 'Ri...",Human,Earth (Replacement Dimension),Male,Alive,5.161649
125,Dr. Wong,https://rickandmorty.fandom.com/wiki/Dr._Wong,"['Beth Smith', 'Jerry Smith', 'Pickle Rick (ch...",Human,Earth (Replacement Dimension),Female,Alive,5.109726
173,Glasses Morty,https://rickandmorty.fandom.com/wiki/Glasses_M...,"['Lizard Morty', 'Slick Morty', 'Morty Smith']",Human,unknown,Male,Alive,5.105075
230,Jessica,https://rickandmorty.fandom.com/wiki/Jessica,"[""Jessica's Grandma"", 'Brad', 'Morty Smith', '...",Cronenberg,Earth (C-137),Female,Alive,5.029209
266,Lizard Morty,https://rickandmorty.fandom.com/wiki/Lizard_Morty,"['Morty Smith', 'Slick Morty', 'Glasses Morty'...",Humanoid,unknown,Male,Alive,5.155842
294,Million Ants,https://rickandmorty.fandom.com/wiki/Million_Ants,"['Supernova', 'Rick Sanchez', 'Alan Rails']",Animal,unknown,Male,Dead,5.083689
309,Mr. Goldenfold,https://rickandmorty.fandom.com/wiki/Mr._Golde...,"['Morty Smith', 'Mrs. Pancakes', 'Rick Sanchez...",Cronenberg,Earth (C-137),Male,Alive,5.195920
322,Mrs. Pancakes,https://rickandmorty.fandom.com/wiki/Mrs._Panc...,"['Mr. Goldenfold', 'Rick Sanchez', 'Morty Smit...",Human,Earth (C-137),Female,Alive,5.306190
358,Principal Vagina,https://rickandmorty.fandom.com/wiki/Principal...,"['Frank Palicky', 'Mr. Meeseeks', 'Lucius Need...",Cronenberg,Earth (C-137),Male,Alive,5.183339
391,Riq IV,https://rickandmorty.fandom.com/wiki/Riq_IV,"['Council of Ricks', 'Rick Sanchez', 'Summer S...",Human,unknown,Male,Dead,4.794841


### VADER Method

In [71]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# calculate the VADER sentiment score for each character

analyzer = SentimentIntensityAnalyzer()

vader_score_list = []

def calculate_VADER_of_a_node(aRow):
    if aRow.Name in list(characters_dialogue.keys()):
        vader_score_list.append(np.mean([analyzer.polarity_scores(aSentence)['compound'] for aSentence in characters_dialogue[aRow.Name]]))
    else:
        vader_score_list.append(None)

_ = df_characters.apply(calculate_VADER_of_a_node, axis=1)

In [72]:
# store in dataframe of characters as a new column
df_characters['VADER_score']=vader_score_list
df_VADER = df_characters[df_characters.VADER_score.notnull()]

In [75]:
df_VADER

,Name,Hyperlink,Relations,Species,Origin,Gender,Status,LabMT_score,VADER_score
64,Brad,https://rickandmorty.fandom.com/wiki/Brad,"['Jessica', 'Morty Smith', 'Summer Smith', 'Ri...",Human,Earth (Replacement Dimension),Male,Alive,5.161649,0.9460
125,Dr. Wong,https://rickandmorty.fandom.com/wiki/Dr._Wong,"['Beth Smith', 'Jerry Smith', 'Pickle Rick (ch...",Human,Earth (Replacement Dimension),Female,Alive,5.109726,0.7573
173,Glasses Morty,https://rickandmorty.fandom.com/wiki/Glasses_M...,"['Lizard Morty', 'Slick Morty', 'Morty Smith']",Human,unknown,Male,Alive,5.105075,0.9206
230,Jessica,https://rickandmorty.fandom.com/wiki/Jessica,"[""Jessica's Grandma"", 'Brad', 'Morty Smith', '...",Cronenberg,Earth (C-137),Female,Alive,5.029209,0.9774
266,Lizard Morty,https://rickandmorty.fandom.com/wiki/Lizard_Morty,"['Morty Smith', 'Slick Morty', 'Glasses Morty'...",Humanoid,unknown,Male,Alive,5.155842,0.7832
294,Million Ants,https://rickandmorty.fandom.com/wiki/Million_Ants,"['Supernova', 'Rick Sanchez', 'Alan Rails']",Animal,unknown,Male,Dead,5.083689,0.4268
309,Mr. Goldenfold,https://rickandmorty.fandom.com/wiki/Mr._Golde...,"['Morty Smith', 'Mrs. Pancakes', 'Rick Sanchez...",Cronenberg,Earth (C-137),Male,Alive,5.195920,0.9694
322,Mrs. Pancakes,https://rickandmorty.fandom.com/wiki/Mrs._Panc...,"['Mr. Goldenfold', 'Rick Sanchez', 'Morty Smit...",Human,Earth (C-137),Female,Alive,5.306190,-0.3387
358,Principal Vagina,https://rickandmorty.fandom.com/wiki/Principal...,"['Frank Palicky', 'Mr. Meeseeks', 'Lucius Need...",Cronenberg,Earth (C-137),Male,Alive,5.183339,0.9974
391,Riq IV,https://rickandmorty.fandom.com/wiki/Riq_IV,"['Council of Ricks', 'Rick Sanchez', 'Summer S...",Human,unknown,Male,Dead,4.794841,0.9725


In [76]:
df_episodes

,Title,Hyperlink,Season_nb,Episode_nb
0,Pilot,https://rickandmorty.fandom.com/wiki/Pilot,1,1
1,Lawnmower Dog,https://rickandmorty.fandom.com/wiki/Lawnmower...,1,2
2,Anatomy Park (Episode),https://rickandmorty.fandom.com/wiki/Anatomy_P...,1,3
3,M. Night Shaym-Aliens!,https://rickandmorty.fandom.com/wiki/M._Night_...,1,4
4,Meeseeks and Destroy,https://rickandmorty.fandom.com/wiki/Meeseeks_...,1,5
5,Rick Potion 9,https://rickandmorty.fandom.com/wiki/Rick_Poti...,1,6
6,Raising Gazorpazorp,https://rickandmorty.fandom.com/wiki/Raising_G...,1,7
7,Rixty Minutes,https://rickandmorty.fandom.com/wiki/Rixty_Min...,1,8
8,Something Ricked This Way Comes,https://rickandmorty.fandom.com/wiki/Something...,1,9
9,Close Rick-counters of the Rick Kind,https://rickandmorty.fandom.com/wiki/Close_Ric...,1,10


In [78]:
df_transcripts.groupby('season no.')

In [82]:
df_transcripts[df_transcripts['season no.'] == 4]

,season no.,episode no.,episode name,name,line


df_transcripts